## Generate inundation maps for various probabilities
Given a set of probabilities and events, this notebook computes the corresponding inundation maps and outputs them to text files in a format javascript can use.

This script recycles lots of code from [Make_Hazard_Curves_and_Maps.ipynb](Make_Hazard_Curves_and_Maps.ipynb).

### Import libraries needed to run notebook

In [8]:
%pylab inline
from __future__ import print_function
from ptha_paths import data_dir, events_dir, js_data_dir
import sys, os

Populating the interactive namespace from numpy and matplotlib


### Set parameters

In [29]:
# The probabilities for which we want to generate inundation maps
probabilities = [1./50, 1./100, 1./200, 1./500, 1./1000, 1./5000, 1./10000]

# A dictionary of sets of events to consider
meta_events         = {}
meta_events['all']  = ['AASZa', 'AASZb', 'AASZc', 'AASZd', 'CSZa', 'CSZb', 'CSZc', 'CSZd', 'CSZe', \
                       'CSZf', 'KmSZa', 'KrSZa', 'SChSZa', 'TOHa']
meta_events['near'] = ['CSZa', 'CSZb', 'CSZc', 'CSZd', 'CSZe', 'CSZf']
meta_events['far']  = ['AASZa', 'AASZb', 'AASZc', 'AASZd', 'KmSZa', 'KrSZa', 'SChSZa', 'TOHa']

# Specify the range of exceedance values in which we are interested (in meters).
# This determines the resolution and range of values in the inundation maps: 
# their values will be in the set linspace(zeta_a,zeta_b,n_zeta).
zeta_a = 0
zeta_b = 12
n_zeta = 121
zeta   = linspace(zeta_a,zeta_b,n_zeta)

# Resolution of the data
nx = 250
ny = 250

# Name of the file where we will output all the inundation maps
out_filename = 'inundation_maps.js'

# Names of some javascript variables we will output
p_inv_name     = 'pInv'
maps_prefix    = "eta_ps_"

### Some information on the events
The naming convention is the name of a subduction zone, followed by an event number, followed by a realization number, but for our purposes we will think of these as independent events.   The subduction zones represented are:
- AASZ: Alaska-Aleutian
- CSZ: Cascadia
- KmSZ: Kamchatka
- KrSZ: Kurils
- SChSZ: Southern Chile
- TOH: Japan / Tohoku

The inundation patterns shown above were computed as part of a project to develop new techniques for probabilistic tsunami hazard assessment (PTHA), and are reprinted from the report 

- *Probabilistic Tsunami Hazard Assessment (PTHA) for Crescent City, CA.*, by Frank I. Gonzalez, Randall J. LeVeque, Loyce M. Adams, Chris Goldfinger, George R. Priest, and Kelin Wang, <http://hdl.handle.net/1773/25916>

They were computed by running the [GeoClaw](http://www.geoclaw.org) tsunami model with the given earthquake source and then monitoring the maximum inundation depth observed at each point on shore over the course of the entire simulation.  (Actually on a fine grid of points covering the region shown above, with horizontal resolution of 1/3 arcsecond, about 10 meters in latitude and 7 meters in longitude).

### Set the annual probability for each event
We create a dictionary storing the probabilities of each event. These were given to us from Randy. We did not come up with them ourselves.

In [14]:
event_prob           = {}
event_prob['AASZa']  = 1./394.
event_prob['AASZb']  = 1./750.
event_prob['AASZc']  = 1./563.
event_prob['AASZd']  = 1./324.
event_prob['CSZa']   = 1./250. * .0125
event_prob['CSZb']   = 1./250. * .0125
event_prob['CSZc']   = 1./250. * .0750
event_prob['CSZd']   = 1./250. * .5000
event_prob['CSZe']   = 1./250. * .1750
event_prob['CSZf']   = 1./250. * .2250
event_prob['KmSZa']  = 1./50.
event_prob['KrSZa']  = 1./167.
event_prob['SChSZa'] = 1./300.
event_prob['TOHa']   = 1./103.

### Define a function to combine two events

In [15]:
def combine_prob(p1,p2):
    """Returns the probability that event 1 or 2 happens"""
    return 1. - (1 - p1) * (1 - p2)

### Create a function that computes inundation for a given probability
A more commonly used map is obtained by fixing a probability (e.g. $p = 0.01$ for a "100-year" flood map) and plotting the maximum depth expected with this annual probability.   

This requires determining, for each grid point `(i,j)`, the largest value of `k` for which `exceed_prob[k]`  $\geq p$. Then the value `zeta[k]` is the largest exceedance value for which the probability is at least $p$.

Recall that `zeta` is defined to be maximum depth of inundation on shore, or maximum height above MHW offshore.

In [16]:
def compute_zeta(p,exceed_prob):

    # create boolean array K with K[i,j,k] == True only where exceed_prob[i,j,k] > p:
    K = exceed_prob > p

    K[:,:,0] = True
    zeta_p = zeros((nx,ny))
    for i in xrange(nx):
        for j in xrange(ny):
            zeta_p[i,j] = zeta[K[i,j,:]][-1]
    return zeta_p

### Define a function for outputting lists in a javascript-friendly format

The main thing this function accomplishes is writing the entries of a list `out_list` to a file object `file_obj` with commas separating each item. For example, `l=[0,1,2,3,4,5]` would be written as `0,1,2,3,4,5`.

In [17]:
def write_list(file_obj, out_list):
    first_flag = True
    for entry in out_list:
        if first_flag:
            file_obj.write(str(entry))
            first_flag = False
        else:
            file_obj.write(',' + str(entry))

### Define a function for outputting javascript array variables

This function writes a python array to a file as a javascript array variable. For example, `write_js_array_variable(f,[0,1,2,3],"seq")` adds the following line to the file object `f`: `const seq = [0,1,2,3]`. That is, it defines a javascript array called `seq` containing the values of the second argument.

In [21]:
def write_js_array_variable(file_obj, out_list, var_name):
    file_obj.write("var %s = [" %(var_name))
    write_list(file_obj,out_list)
    file_obj.write("];\n")

### Form and save inundation maps

In [31]:
# Get location of file where we'll output the results
out_file = os.path.join(js_data_dir,out_filename)

# Loop over the meta events
first_event = True
for meta_event, events in meta_events.iteritems():
    
    # Compute the combined probability of exceeding each exceedance value
    exceed_prob = zeros((nx,ny,n_zeta))
    
    # Loop over all events and update exceed_prob at each grid point by combining
    # current value with the probability Pk of this event:

    for event in events:
        event_dir = os.path.join(events_dir, event)
        hmax_file = os.path.join(event_dir, 'h_eta_small.npy')
        hmax      = load(hmax_file)
        Hmax      = hmax.reshape((nx,ny),order='F')
        for k in xrange(n_zeta):
            Pk = exceed_prob[:,:,k]  # probabilities at all points for one exceedance value zeta_k
            exceed_prob[:,:,k] = where(Hmax > zeta[k], combine_prob(event_prob[event],Pk), Pk)
    
    # Get the inundation array associated with each probability and write to out_file
    for probability in probabilities:
        # Get the inundation array
        inundation_array = compute_zeta(probability,exceed_prob)
        
        # Write to a variable in a text file
        var_name = "%s_eta_pInv%d" %(meta_event, int(1./probability))
        header = "var %s = {\"width\":250,\"height\":250,\"values\":[" %(var_name)
        footer = "]};\n"
        
        # Overwrite old file
        if first_event:
            with open(out_file,'w') as f:
                f.write(header)
                write_list(f,inundation_array.ravel())
                f.write(footer)
#                 write_js_array_variable(f,inundation_array.ravel(),var_name)
                first_event = False
        else:
            with open(out_file,'a') as f:
                f.write(header)
                write_list(f,inundation_array.ravel())
                f.write(footer)
#                 write_js_array_variable(f,inundation_array.ravel(),var_name)
        
#         with open(out_file,'w') as f:
#             f.write(header)
#             first_flag = True
#             for entry in inundation_array.ravel():
#                 if first_flag:
#                     f.write(str(entry))
#                     first_flag = False
#                 else:
#                     f.write(',' + str(el))
#             f.write(footer)
            
    
        
    print("Done computing inundation map for %s events." % meta_event)

# Append some extra variables to the end of the file
with open(out_file,'a') as f:
    # Array containing strings of probabilities
    p_inv = [str(int(1./p)) for p in probabilities]
    write_js_array_variable(f,p_inv,p_inv_name)
    
    # Arrays containing the inundation map variables for the different subsets of events
    for meta_event in meta_events:
        maps = ["%s_eta_pInv%d" %(meta_event,int(1./probability)) for probability in probabilities]
        write_js_array_variable(f, maps, maps_prefix + meta_event)
    
#     # Array containing all the near event inundation maps
#     near_maps = ["near_eta_pInv%d" %(int(1./probability)) for probability in probabilities]
#     write_js_array_variable(f,near_maps,near_maps_name)
    
#     # Array containing all the far event inundation maps
#     far_maps = ["near_eta_pInv%d" %(int(1./probability)) for probability in probabilities]
#     write_js_array_variable(f,far_maps,far_maps_name)
    
#     # Array containing all the inundation maps
#     all_maps = ["all_eta_pInv%d" %(int(1./probability)) for probability in probabilities]
#     write_js_array_variable(f,all_maps,all_maps_name)
    

Done computing inundation map for far events.
Done computing inundation map for all events.
Done computing inundation map for near events.
